In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials

from datetime import datetime, timedelta
from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager

import polars as pl
import pandas as pd

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

setup_logging(
    level=logging.DEBUG,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

In [2]:
def get_trade_day() -> str:
    trade_day = datetime.now()
    while trade_day.weekday() >= 5:
        trade_day += timedelta(days=1)

    return trade_day.strftime("%y%m%d")

In [3]:
yy_mm_dd = get_trade_day()

symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    "BCH/USD:CXTALP",
    "ETH/USD:CXTALP",
    # f".SPXW{yy_mm_dd}C5915",
    # f".SPXW{yy_mm_dd}C5910",
    # f".SPXW{yy_mm_dd}P5910",
    # f".SPXW{yy_mm_dd}P5905",
]

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)

# Open WebSocket connection
dxlink = DXLinkManager()
await dxlink.open(credentials)
await dxlink.subscribe(symbols)

In [ ]:
for queue in sorted(dxlink.queues):
    print(queue, dxlink.queues[queue].qsize())

In [ ]:
for task in sorted(asyncio.all_tasks(), key=lambda x: x.get_name()):
    # if task.get_name().startswith("Task"):
    print(f"Task name: {task.get_name()}")
    print(f"Task coro: {task.get_coro()}")
    print(f"Task frame: {task.get_coro().cr_frame}")
    print(f"Task frame locals: {task.get_coro().cr_frame.f_locals}")
    print("---")

In [ ]:
await dxlink.close()

In [ ]:
trades_df = dxlink.router.handler[Channels.Trade].processors["feed"].df
trades_df

In [ ]:
new_symbols = [
    f".SPXW{yy_mm_dd}C6110",
    f".SPXW{yy_mm_dd}C6105",
    f".SPXW{yy_mm_dd}P6105",
    f".SPXW{yy_mm_dd}P6100",
]
await dxlink.subscribe(new_symbols)

In [ ]:
trades = dxlink.router.handler[Channels.Trade].processors["feed"].df
# trades.loc[trades["eventSymbol"] == "SPX"]
trades

In [11]:
# df = dxlink.router.handler[Channels.Quote].processors["feed"].df
spx = (
    dxlink.router.handler[Channels.Quote]
    .processors["feed"]
    .pl.filter(pl.col("eventSymbol") == "SPX")
    .tail(1)
)
# (df["bidPrice"] + df["askPrice"]) / 2

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].pl

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df
# quotes.loc[quotes["eventSymbol"] == "SPX"]["bid-price"]

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [18]:
# (
#     df.with_columns(pl.col("strike-price").cast(pl.Float32(), strict=False))
#     .with_columns(
#         pl.col("option-type").map_elements(lambda x: OptionType(x).name, return_dtype=str)
#     )
#     .filter((pl.col("expiration-date") == trade_date))
#     .filter((pl.col("strike-price") >= 5700))
#     .select(["strike-price", "option-type", "streamer-symbol"])
#     .sort("strike-price")
#     .head(10)
# )

In [ ]:
def round_to_nearest_5(value: float) -> float:
    return round(value / 5) * 5


round_to_nearest_5(1002.123)